In [1]:
from PIL import Image
import os
import shutil
import glob
import pandas as pd
#import matplotlib
#matplotlib.use("TkAgg")  # Do this before importing pyplot!
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [2]:
ar=np.arange(62)
fldrs=[]
for i in range(10):
    fldrs.append(str('Training/'+'0000'+str(ar[i])))
for i in range(10,62):
    fldrs.append(str('Training/'+'000'+str(ar[i])))

In [4]:
#paths=glob.glob('./'+newpath+'/*.csv')
def read_image_paths(newpath):
    try:
        if os.path.exists(newpath):
           # print('\n Such path exists \n')
            paths=glob.glob(newpath+'/*.ppm')
            #print(paths)
        else:
            print('Path does not exist') 
        return paths
        #return {i:pd.read_csv(i,index_col=['date'],parse_dates=['date']) for i in paths},paths #an error occurs when index_col is wrong 
    except Exception:
        print('Error reading stock data')
        return -1
def read_image_info(newpath):
    try:
        if os.path.exists(newpath):
            #print('\n Such path exists \n')
            paths=glob.glob(newpath+'/*.csv')
            #im_paths=glob.glob(newpath+'/*.ppm')
            #print(paths)
        else:
            print('Path does not exist') 
        df=pd.read_csv(paths[0],sep=';',header=0)
        #df['Path']=pd.Series(im_paths)
        return df#pd.read_csv(paths[0],sep=';',header=0) 
    except Exception:
        print('Error reading stock data',Exception)
        return -1
def read_all_image_info(fldrs):
    # it will be better to process exceptions
    #paths=[]
    res=read_image_info(fldrs[0])
    res['Path']=pd.Series(add_full_path(fldrs[0],res)) ########################################################Added full path
    for i in range(len(fldrs)-1):
        df2=read_image_info(fldrs[i+1])
        df2['Path']=pd.Series(add_full_path(fldrs[i+1],df2))#############################################Added full path
        frames=[res,df2]
        res=pd.concat(frames)
        res=res.reset_index(drop=True)       
    return res

def to_gray(color_img):
    gray=cv2.cvtColor(color_img,cv2.COLOR_BGR2GRAY)
    return gray

def gen_sift_features(gray_img):
    sift = cv2.xfeatures2d.SIFT_create()
    # kp is the keypoints
    #
    # desc is the SIFT descriptors, they're 128-dimensional vectors
    # that we can use for our final features
    kp, desc = sift.detectAndCompute(gray_img, None)
    return kp, desc

def show_sift_features(gray_img, color_img, kp):
    return plt.imshow(cv2.drawKeypoints(gray_img, kp, color_img.copy()))
def concat_frames(df1,df2):
    frames=[df1,df2]
    res=pd.concat(frames)
    res=res.reset_index(drop=True)
    return res

In [5]:
def proper_sized_image(im):
    h,w=im.shape #first height second width
    if w>minw:
        im=cv2.resize(im,(minw,h))
    h,w=im.shape
    if h>minh:
        im=cv2.resize(im,(w,minh))
    h,w=im.shape
    if minw>w or minh>h:
        im=cv2.resize(im,(minw, minh),interpolation=cv2.INTER_NEAREST)        
    return im
def read_image(newpath):
        try:
            if os.path.exists(newpath):
                1==1
                # print('\n Such path exists \n')
            else:
                print('Path does not exist') 
            image=cv2.imread(newpath)
            gray=to_gray(image)
            #image=proper_sized_image(image)
            gray=proper_sized_image(gray)
            #gray = cv2.GaussianBlur(gray, (3, 3), 0)
            return image ,gray
        except Exception:
            print('Error reading stock data')
            return -1
def im_show(im):
    cv2.imshow('image',im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def int_ar(ar):
    mas=list(map(lambda x: int(x), ar))
    return mas
def label_from_path(path):
    return info_all[info_all['Path']==path]['ClassId'].tolist()[0]
def first_picture__from_label(df,label):
    try:
        path=df[df['ClassId']==label]['Path'].tolist()[0]
        im=read_image(path)
        im_show(im[1])
        return 0
    except Exception:
        return -1
def add_full_path(folder,df):
    return list(map(lambda x:str(folder)+'/'+str(x),df['Filename'].tolist()))

# Define functions to create bag of words(to merge sift descriptors)

In [6]:
def preprocess1_image(path):
    image=read_image(path)
    im_kp, im_desc=gen_sift_features(image[1])
    return im_desc
def preprocess_images(paths):
    #labels=[]
    i=0
    for p in paths:
        if i==0:
            bag=preprocess1_image(p)
            #print(len(bag))
            i+=1
        else:
            #try: 
            bag=np.vstack((bag,preprocess1_image(p)))
            #print(len(bag))
    return bag
           # except Exception:
            
            #labels.append(label_from_path(p))
    

# Cluster bag of words

In [7]:
def cluster(bag_of_words,n_clusters):
    try:
        if len(bag_of_words)>n_clusters:
            kmeans=KMeans(n_clusters=n_clusters,random_state=0).fit(bag_of_words)
            return kmeans # define only for 1?tures1=kmeans.labels_
        else:
            print('not enough features to classify: ',len(bag_of_words))
    except Exception:
        print("bag of words exception occured: ",Exception)
        return -1

# Predict histogram for an image

In [8]:
def image_histogram(kmeans,image_path,n_clusters):
    litter,image=read_image(image_path)
    kp,image_descriptors=gen_sift_features(image)
    set_of_clusters=kmeans.predict(image_descriptors)
    hist,bins=np.histogram(set_of_clusters,n_clusters)
    #plt.hist(set_of_clusters,bins=n_clusters)
    return hist
def images_to_histograms(kmeans,paths,n_clusters):
    try:
        hists=[]
        labels=[]
        for p in paths:
            hists.append(image_histogram(kmeans,p,n_clusters))
            labels.append(label_from_path(p))
        return np.array(hists) , np.array(labels)
    except Exception:
        print('histograms exception occured: ',Exception)

# Set info

In [9]:
info_all=read_all_image_info(fldrs)
minh=int(round(info_all['Height'].mean()))
minw=int(round(info_all['Width'].mean()))
print (minw,minh)

125 110


# testing image from label function

In [10]:
first_picture__from_label(info_all,61)

0

# Sift application

## make small set 

In [11]:
def make_small_set(inform,l1,l2):
    return concat_frames(inform[inform['ClassId']==l1],inform[inform['ClassId']==l2])

In [12]:
first_ar=make_small_set(info_all,39,2)

## test bag of words

In [13]:
bag_of_words=preprocess_images(first_ar['Path'])

In [14]:
%%time
n=150
kmeans=cluster(bag_of_words,n)
print(image_histogram(kmeans,first_ar['Path'].tolist()[0],n))

[ 2  1  0  0  2  1  0  1  0  0  0  3  1  1  4  0  2  0  1  0  0  2  1  1
  1  1  0  1  1  0  0  1  1  0  1  0  2  0  2  0  1  1  1  1  1  0  1  0
  0  0  1  1  2  1  0  0  0  1  1  8  0  1  0  0  0  0  0  0  2  0  2  1
  1  1  1  0  0  0  0  1  2 17  0  2  4  3  0  1  0  1  2  1  0  0  0  2
  0  0  0  0  0  2  0  0  3  1  1  1  1  1  0  0  0  0  0  2  0  0  1  2
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  1
  0  0  2  0  0  2]


In [15]:
histograms,labels=images_to_histograms(kmeans,first_ar['Path'].tolist(),n)

In [16]:
len(histograms)

209

## test data and labeling 

In [17]:
X=histograms
y=labels

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(pred)

[39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39
 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39  2
 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39 39]


In [24]:
accuracy_score(y_test,pred)

0.9565217391304348

# Define function model_SIFT_SVC

In [25]:
def prepare_data_for_classification(first_ar,n):
    bag_of_words=preprocess_images(first_ar['Path'])
    kmeans=cluster(bag_of_words,n)
    X,y=images_to_histograms(kmeans,first_ar['Path'].tolist(),n)  
    return X,y
def model_SIFT_SVC(first_ar,n):
    X,y=prepare_data_for_classification(first_ar,n)
    #bag_of_words=preprocess_images(first_ar['Path'])
    #kmeans=cluster(bag_of_words,n)
    #X,y=images_to_histograms(kmeans,first_ar['Path'].tolist(),n)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,stratify=y)
    clf = SVC(gamma='auto')
    clf.fit(X_train, y_train)
    pred=clf.predict(X_test)
    acc_score=accuracy_score(y_test,pred)
    print(acc_score)
    return acc_score,clf   
def optimize_n_clusters(first_ar,nmin,nmax,step=15):
    mscore=0
    i=nmin
    m_n_clusters=i
    while i in range(nmin,nmax):
        score,classifier=model_SIFT_SVC(first_ar,i)
        if score>mscore:
            mscore=score
            m_n_clusters=i
        i+=step
    return m_n_clusters,mscore   
def make_df_several_labls(inform,labels):# it seems to work well
    df=inform[inform['ClassId']==labels[0]]
    for i in range(1,len(labels)):
        df=concat_frames(df,inform[inform['ClassId']==labels[i]])
    return df    

# let's find out what classes are most common

In [26]:
info_all['ClassId'].value_counts()[:5].index.tolist()

[22, 32, 38, 61, 40]

# Main part of the code

In [29]:
%%time
n=200
#2 classes
first_ar=make_small_set(info_all,32,22)
res=model_SIFT_SVC(first_ar,n)

0.8427947598253275
Wall time: 7min 8s


In [30]:
n=200
labels=info_all['ClassId'].value_counts()[:5].index.tolist() #if labels are 0,4,6 the best n=200
data=make_df_several_labls(info_all,labels)
#first_ar=make_small_set(info_all,32,28)#22,32 (39,7) n=250

In [31]:
%%time
res=model_SIFT_SVC(data,n)

0.6
Wall time: 19min 59s
